In [1]:
#THESE SCRIPTS TAKE US FROM RAW DATA TO THE TABLES WE HAVE IN OUR SCHEMA - SEE movie_db_model.png

c:\python37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import pandas as pd
import numpy as np
from itertools import chain

In [3]:
##BEGIN CREATION OF ACTOR_LOOKUP TABLE
df_name_basics = pd.read_csv('raw_data/name_basics.tsv', sep='\t')
print(list(df_name_basics))
##Bring in titles
df_title = pd.read_csv('app_data/title.csv')
title_ids = pd.Series(df_title['title_id'])

##Bring in actors
df_actors = pd.read_csv('app_data/actor.csv')
actor_ids = pd.Series(df_actors['actor_id'])


['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']


In [4]:
#one row for each comma separated cell value, inspired by:
#https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df_name_basics['knownForTitles'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
                        #Repeating columns
df_name_basics_flat = pd.DataFrame({'nconst': np.repeat(df_name_basics['nconst'], lens),
                    'primaryName': np.repeat(df_name_basics['primaryName'], lens),
                    'birthYear': np.repeat(df_name_basics['birthYear'], lens),
                    'deathYear': np.repeat(df_name_basics['deathYear'], lens),
                    'primaryProfession': np.repeat(df_name_basics['primaryProfession'], lens),
                    #Newly split column
                    'knownForTitles': chainer(df_name_basics['knownForTitles'])})


print(df_name_basics_flat[:20])



      nconst      primaryName birthYear deathYear  \
0  nm0000001     Fred Astaire      1899      1987   
0  nm0000001     Fred Astaire      1899      1987   
0  nm0000001     Fred Astaire      1899      1987   
0  nm0000001     Fred Astaire      1899      1987   
1  nm0000002    Lauren Bacall      1924      2014   
1  nm0000002    Lauren Bacall      1924      2014   
1  nm0000002    Lauren Bacall      1924      2014   
1  nm0000002    Lauren Bacall      1924      2014   
2  nm0000003  Brigitte Bardot      1934        \N   
2  nm0000003  Brigitte Bardot      1934        \N   
2  nm0000003  Brigitte Bardot      1934        \N   
2  nm0000003  Brigitte Bardot      1934        \N   
3  nm0000004     John Belushi      1949      1982   
3  nm0000004     John Belushi      1949      1982   
3  nm0000004     John Belushi      1949      1982   
3  nm0000004     John Belushi      1949      1982   
4  nm0000005   Ingmar Bergman      1918      2007   
4  nm0000005   Ingmar Bergman      1918      2

In [5]:
#strip down only to necessary columns and output to app_data folder
df_actor_lookup = df_name_basics_flat[['nconst','knownForTitles']]

#Limit to only titles of interest
df_actor_lookup = df_actor_lookup.loc[df_actor_lookup['knownForTitles'].isin(title_ids)]

#Limit to only actors in our actors table
df_actor_lookup = df_actor_lookup.loc[df_actor_lookup['nconst'].isin(actor_ids)]

print(df_actor_lookup.head())

        nconst knownForTitles
174  nm0000175      tt0111161
208  nm0000209      tt0111161
292  nm0000293      tt0944947
316  nm0000317      tt0111161
941  nm0000943      tt3572718


In [7]:
#Rename columns
df_actor_lookup.columns=['actor_id', 'title_id']
print(df_actor_lookup.head())

      actor_id   title_id
174  nm0000175  tt0111161
208  nm0000209  tt0111161
292  nm0000293  tt0944947
316  nm0000317  tt0111161
941  nm0000943  tt3572718


In [14]:
#Add primary key column
# df_actor_lookup = df_actor_lookup.reset_index()
# df_actor_lookup = df_actor_lookup.reset_index()
df_actor_lookup = df_actor_lookup[['level_0', 'actor_id', 'title_id']]
print(df_actor_lookup.head())

   level_0   actor_id   title_id
0        0  nm0000175  tt0111161
1        1  nm0000209  tt0111161
2        2  nm0000293  tt0944947
3        3  nm0000317  tt0111161
4        4  nm0000943  tt3572718


In [16]:
df_actor_lookup.columns=['actor_lookup_id', 'actor_id', 'title_id']
df_actor_lookup.to_csv('app_data/actor_lookup.csv', index=False)

In [10]:
print(list(df_actor_lookup))

['actor_lookup_id', 'actor_id', 'title_id']
